In [51]:
import pandas as pd
import requests
import json
from datetime import datetime, timedelta, date, time

my_openweather_api_key = 88888888888888888888888
my_RapidAPI_key = 888888888888888888

In [24]:
# get timestamp in correct format for tomorrow 8 AM until 8 PM
def get_tomorrow():
    date_tomorrow = date.today() + timedelta(days=1)
    time_start = time(hour=0, minute=0, second=0)
    time_middle = time(hour=12, minute=0, second=0)
    time_end = time(hour=23, minute=59, second=59)

    datetime_start = datetime.combine(date_tomorrow, time_start).strftime('%Y-%m-%dT%H:%M')
    datetime_middle = datetime.combine(date_tomorrow, time_middle).strftime('%Y-%m-%dT%H:%M')
    datetime_end = datetime.combine(date_tomorrow, time_end).strftime('%Y-%m-%dT%H:%M')
    return datetime_start,datetime_middle,datetime_end

In [50]:
def weather_forcast(list_of_cities):
   
    cities_list  = []
    weather_list = []
    
    for city in list_of_cities :
        city_r = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={my_openweather_api_key}&units=metric")
        city_j = city_r.json()
        
        cities_list.append(city_j['city'])
        

        for detail in city_j['list']:
            weather_details_dic = {}
            weather_details_dic["city_id"] = city_j['city']['id']
            weather_details_dic["city_name"] = city_j['city']['name']
            weather_details_dic["temp"] = detail['main']['temp']
            weather_details_dic["temp_min"] = detail['main']['temp_min']
            weather_details_dic["temp_max"] = detail['main']['temp_max']
            weather_details_dic["feels_like"] = detail['main']['feels_like']
            weather_details_dic["pressure"] = detail['main']['pressure']
            weather_details_dic["humidity"] = detail['main']['humidity']
            weather_details_dic["weather"] = detail['weather'][0]['main']
            weather_details_dic["weather_description"] = detail['weather'][0]['description']
            weather_details_dic["clouds"] = detail['clouds']['all']
            weather_details_dic["wind_speed"] = detail['wind']['speed']
            weather_details_dic["date_time"] = detail['dt_txt']
            weather_list.append(weather_details_dic)
    
    
    df_weather = pd.DataFrame(weather_list)
    dt_start,dt_middle,dt_end = get_tomorrow()
    df_weather['date_time'] = pd.to_datetime(df_weather['date_time'])
    df_weather = df_weather.loc[lambda df_ : (df_['date_time'] >= dt_start) &  (df_['date_time'] <= dt_end)]
    
    df_cities = pd.json_normalize(cities_list)
    
    
    return df_weather,df_cities

In [35]:
df1, df2 = weather_forcast(['LONDON'])

In [37]:
df2

,id,name,country,population,timezone,sunrise,sunset,coord.lat,coord.lon
0,2643743,London,GB,1000000,3600,1659414327,1659469679,51.5085,-0.1257


In [49]:
def get_citys_airports(city_name, country_iso2):
    
	url = "https://aerodatabox.p.rapidapi.com/airports/search/term"

	querystring = {"q":city_name,"limit":"10"}

	headers = {
		"X-RapidAPI-Key": my_RapidAPI_key,
		"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
	}

	airports_r = requests.request("GET", url, headers=headers, params=querystring)
	airports_j = airports_r.json()

	df_airports =pd.json_normalize(airports_j['items'])
	df_airports = df_airports.loc[lambda df_ : df_['countryCode'] == country_iso2]
	
	return df_airports

In [47]:
temp = get_citys_airports('LONDON','GB')

In [48]:
# right_airports
temp

,icao,iata,name,shortName,municipalityName,countryCode,location.lat,location.lon
0,EGSS,STN,"London, London Stansted",Stansted,London,GB,51.8850,0.234999
1,EGLL,LHR,"London, London Heathrow",Heathrow,London,GB,51.4706,-0.461941
2,EGLC,LCY,"London, London City",City,London,GB,51.5053,0.055277
3,EGKK,LGW,"London, London Gatwick",Gatwick,London,GB,51.1481,-0.190277
4,EGGW,LTN,"London, London Luton",Luton,London,GB,51.8747,-0.368333
9,EGKB,BQH,"London, London Biggin Hill",Biggin Hill,London,GB,51.3308,0.032499


In [80]:
# make api call to receive flight information

dt_start,dt_middle,dt_end = get_tomorrow()
icao = 'EDDB'
url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{dt_start}/{dt_middle}"

querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"false","withCargo":"false","withPrivate":"false","withLocation":"false"}

headers = {
	"X-RapidAPI-Key": my_RapidAPI_key,
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

arrivals_r = requests.request("GET", url, headers=headers, params=querystring)

arrivals_j = arrivals_r.json()
arrivals_df = pd.json_normalize(arrivals_j['arrivals']).sort_values(by='movement.scheduledTimeLocal')

# clean df
arrivals_df = arrivals_df.loc[lambda df_ : df_['isCargo']== False]
arrivals_df = arrivals_df[['number', 'status', 
                'movement.airport.name','movement.airport.icao', 'movement.airport.iata', 
                'movement.scheduledTimeLocal', 'movement.scheduledTimeUtc', 'movement.terminal',
                'airline.name', 'aircraft.model']]
rename_dict= {'number':'flight_NO', 'status':'flight_status', 
                'movement.airport.name' : 'origin_ap_name','movement.airport.icao' : 'origin_ap_icao', 'movement.airport.iata':'origin_ap_iata', 
                'movement.scheduledTimeLocal' : 'scheduledTimeLocal', 'movement.scheduledTimeUtc' : 'scheduledTimeUtc', 'movement.terminal':'terminal',
                'airline.name':'airline_name', 'aircraft.model' :'aircraf_model'}
arrivals_df.rename(columns=rename_dict , inplace=True)


# flight_data_df.columns = (['flight_nr', 'origin_airport_city', 'scheduled_arrival_time_local', 
#     'scheduled_arrival_time_utc', 'terminal', 'flight_type', 'aircraft_model', 'airline', 
#     'origin_airport_icao', 'origin_airport_iata'])

In [85]:
tmpp=arrivals_df
rename_dict= {'number':'flight_NO', 'status':'flight_status', 
                'movement.airport.name' : 'origin_ap_name','movement.airport.icao' : 'origin_ap_icao', 'movement.airport.iata':'origin_ap_iata', 
                'movement.scheduledTimeLocal' : 'scheduledTimeLocal', 'movement.scheduledTimeUtc' : 'scheduledTimeUtc', 'movement.terminal':'terminal',
                'airline.name':'airline_name', 'aircraft.model' :'aircraf_model'}
tmpp.rename(columns=rename_dict).index_reset()


AttributeError: 'DataFrame' object has no attribute 'index_reset'

In [77]:

f3= pd.json_normalize(arrivals_j['arrivals']).sort_values(by='movement.scheduledTimeLocal')

In [78]:
f3

,number,status,codeshareStatus,isCargo,movement.airport.name,movement.scheduledTimeLocal,movement.scheduledTimeUtc,movement.terminal,movement.quality,aircraft.model,airline.name,movement.airport.icao,movement.airport.iata,aircraft.reg,aircraft.modeS,callSign
51,XQ 966,Unknown,IsOperator,False,İzmir,2022-08-03 05:45+02:00,2022-08-03 03:45Z,1,[Basic],Boeing 737-800,SunExpress,LTBJ,ADB,NaN,NaN,NaN
7,EW 9048,Unknown,IsOperator,False,Duesseldorf,2022-08-03 07:35+02:00,2022-08-03 05:35Z,1,[Basic],Airbus A220-300,Eurowings,EDDL,DUS,NaN,NaN,GWI52FP
12,EW 2002,Unknown,IsOperator,False,Stuttgart,2022-08-03 07:40+02:00,2022-08-03 05:40Z,1,[Basic],Airbus A319,Eurowings,EDDS,STR,NaN,NaN,NaN
27,UA 962,Unknown,IsOperator,False,Newark,2022-08-03 07:50+02:00,2022-08-03 05:50Z,1,[Basic],Boeing 767-400,United,KEWR,EWR,NaN,NaN,NaN
3,LH 170,Unknown,Unknown,False,Frankfurt-am-Main,2022-08-03 07:55+02:00,2022-08-03 05:55Z,1,[Basic],Airbus A320,Lufthansa,EDDF,FRA,D-AIUG,3C66A7,NaN
44,W6 4275,Unknown,Unknown,False,Tuzla,2022-08-03 08:00+02:00,2022-08-03 06:00Z,0,[Basic],Airbus A320,Wizz Air,LQTZ,TZL,NaN,NaN,NaN
54,TR 734,Unknown,Unknown,False,Singapore,2022-08-03 08:00+02:00,2022-08-03 06:00Z,1,[Basic],Boeing 787-8,Scoot,WSSS,SIN,NaN,NaN,NaN
23,LG 9471,Unknown,Unknown,False,Luxembourg,2022-08-03 08:05+02:00,2022-08-03 06:05Z,1,[Basic],Bombardier Dash 8 Q400 / DHC-8-400,Luxair,ELLX,LUX,NaN,NaN,NaN
11,DX 121,Unknown,Unknown,False,Saarbrücken,2022-08-03 08:10+02:00,2022-08-03 06:10Z,0,[Basic],ATR 42-300,Danish Air,EDDR,SCN,NaN,NaN,NaN
8,EW 9050,Unknown,Unknown,False,Duesseldorf,2022-08-03 08:15+02:00,2022-08-03 06:15Z,1,[Basic],Airbus A319,Eurowings,EDDL,DUS,NaN,NaN,GWI9050


In [75]:
f2

,number,status,codeshareStatus,isCargo,movement.airport.icao,movement.airport.iata,movement.airport.name,movement.scheduledTimeLocal,movement.actualTimeLocal,movement.scheduledTimeUtc,movement.actualTimeUtc,movement.terminal,movement.quality,aircraft.reg,aircraft.modeS,aircraft.model,airline.name,callSign
0,4Y 133,Expected,IsOperator,False,FYWH,WDH,Windhoek,2022-08-03 05:25+02:00,2022-08-03 05:25+02:00,2022-08-03 03:25Z,2022-08-03 03:25Z,1,"[Basic, Live]",D-AFYR,3C5B32,Airbus A330,Airbus Transport International,NaN
14,LH 595,Expected,IsOperator,False,DNAA,ABV,Abuja,2022-08-03 05:30+02:00,2022-08-03 05:30+02:00,2022-08-03 03:30Z,2022-08-03 03:30Z,1,"[Basic, Live]",D-AIKK,3C656B,Airbus A330-300,Lufthansa,NaN
117,LH 573,Expected,IsOperator,False,FAOR,JNB,Jo'anna,2022-08-03 05:35+02:00,2022-08-03 05:35+02:00,2022-08-03 03:35Z,2022-08-03 03:35Z,1,"[Basic, Live]",D-ABYO,3C4B2F,Boeing 747,Lufthansa,NaN
90,LH 591,Expected,IsOperator,False,HKJK,NBO,Nairobi,2022-08-03 05:40+02:00,2022-08-03 05:40+02:00,2022-08-03 03:40Z,2022-08-03 03:40Z,1,"[Basic, Live]",D-AIKL,3C656C,Airbus A330,Lufthansa,NaN
16,LH 1327,Expected,IsOperator,False,DTTA,TUN,Tunis,2022-08-03 05:40+02:00,2022-08-03 05:40+02:00,2022-08-03 03:40Z,2022-08-03 03:40Z,1,"[Basic, Live]",D-AIDH,3C6488,Airbus A321,Lufthansa,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,LH 859,Expected,IsOperator,False,ENGM,OSL,Oslo,2022-08-03 12:00+02:00,2022-08-03 12:00+02:00,2022-08-03 10:00Z,2022-08-03 10:00Z,1,"[Basic, Live]",D-AINY,3C65D9,Airbus A320 NEO,Lufthansa,NaN
74,LH 103,Expected,IsOperator,False,EDDM,MUC,Munich,2022-08-03 12:00+02:00,2022-08-03 12:00+02:00,2022-08-03 10:00Z,2022-08-03 10:00Z,1,"[Basic, Live]",D-AIRW,3C6657,Airbus A321,Lufthansa,NaN
162,LH 231,Expected,IsOperator,False,LIRF,FCO,Rome,2022-08-03 12:00+02:00,2022-08-03 12:00+02:00,2022-08-03 10:00Z,2022-08-03 10:00Z,1,"[Basic, Live]",D-AIZZ,3C675A,Airbus A320 (Sharklets),Lufthansa,NaN
72,LH 393,Expected,IsOperator,False,ELLX,LUX,Luxembourg,2022-08-03 12:00+02:00,2022-08-03 12:00+02:00,2022-08-03 10:00Z,2022-08-03 10:00Z,1,"[Basic, Live]",D-ACNE,3C4DC5,Canadair CRJ 900NG,Lufthansa,NaN


In [73]:
f1

,number,status,codeshareStatus,isCargo,movement.airport.icao,movement.airport.iata,movement.airport.name,movement.scheduledTimeLocal,movement.actualTimeLocal,movement.scheduledTimeUtc,movement.actualTimeUtc,movement.terminal,movement.quality,airline.name,aircraft.model
0,FR 8165,Expected,Unknown,False,LEMG,AGP,Málaga,2022-08-03 00:00+01:00,2022-08-03 00:00+01:00,2022-08-02 23:00Z,2022-08-02 23:00Z,1,"[Basic, Live]",Ryanair,NaN
1,FR 8348,Expected,Unknown,False,LPPR,OPO,Porto,2022-08-03 00:00+01:00,2022-08-03 00:00+01:00,2022-08-02 23:00Z,2022-08-02 23:00Z,1,"[Basic, Live]",Ryanair,NaN
2,LS 1482B,Expected,Unknown,False,NaN,NaN,Kos,2022-08-03 00:00+01:00,2022-08-03 00:00+01:00,2022-08-02 23:00Z,2022-08-02 23:00Z,1,"[Basic, Live]",Jet2,NaN
29,RK 3034,Expected,IsOperator,False,GMAD,AGA,Agadir,2022-08-03 00:05+01:00,2022-08-03 00:05+01:00,2022-08-02 23:05Z,2022-08-02 23:05Z,1,"[Basic, Live]",Ryanair UK,Boeing 737-800
45,FR 8583,Expected,IsOperator,False,LGTS,SKG,Thessaloniki,2022-08-03 00:10+01:00,2022-08-03 00:10+01:00,2022-08-02 23:10Z,2022-08-02 23:10Z,1,"[Basic, Live]",Ryanair,Boeing 737-800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,TK 4320,Unknown,Unknown,False,LTAI,AYT,Antalya,2022-08-03 11:30+01:00,NaN,2022-08-03 10:30Z,NaN,NaN,[Basic],Turkish,Boeing 737-800
60,FR 305,Unknown,Unknown,False,LOWS,SZG,Salzburg,2022-08-03 11:40+01:00,NaN,2022-08-03 10:40Z,NaN,NaN,[Basic],Ryanair,Boeing 737-800
53,FR 683,Unknown,Unknown,False,ESOW,VST,Stockholm / Västerås,2022-08-03 11:40+01:00,NaN,2022-08-03 10:40Z,NaN,NaN,[Basic],Ryanair,Boeing 737-800
73,FR 2673,Unknown,Unknown,False,LIRA,CIA,Roma,2022-08-03 11:55+01:00,NaN,2022-08-03 10:55Z,NaN,NaN,[Basic],Ryanair,Boeing 737-800


In [69]:
arrivals_j['arrivals'][-1]

{'movement': {'airport': {'icao': 'WSSS', 'iata': 'SIN', 'name': 'Singapore'},
  'scheduledTimeLocal': '2022-08-03 06:45+02:00',
  'actualTimeLocal': '2022-08-03 06:45+02:00',
  'scheduledTimeUtc': '2022-08-03 04:45Z',
  'actualTimeUtc': '2022-08-03 04:45Z',
  'terminal': '1',
  'quality': ['Basic', 'Live']},
 'number': 'SQ 26',
 'status': 'Expected',
 'codeshareStatus': 'IsOperator',
 'isCargo': False,
 'aircraft': {'model': 'Airbus A380-800'},
 'airline': {'name': 'Singapore'}}

In [72]:
pd.json_normalize(arrivals_j['arrivals']).sort_values(by='movement.scheduledTimeLocal')

,number,status,codeshareStatus,isCargo,movement.airport.icao,movement.airport.iata,movement.airport.name,movement.scheduledTimeLocal,movement.actualTimeLocal,movement.scheduledTimeUtc,movement.actualTimeUtc,movement.terminal,movement.quality,aircraft.reg,aircraft.modeS,aircraft.model,airline.name,callSign
0,4Y 133,Expected,IsOperator,False,FYWH,WDH,Windhoek,2022-08-03 05:25+02:00,2022-08-03 05:25+02:00,2022-08-03 03:25Z,2022-08-03 03:25Z,1,"[Basic, Live]",D-AFYR,3C5B32,Airbus A330,Airbus Transport International,NaN
14,LH 595,Expected,IsOperator,False,DNAA,ABV,Abuja,2022-08-03 05:30+02:00,2022-08-03 05:30+02:00,2022-08-03 03:30Z,2022-08-03 03:30Z,1,"[Basic, Live]",D-AIKK,3C656B,Airbus A330-300,Lufthansa,NaN
117,LH 573,Expected,IsOperator,False,FAOR,JNB,Jo'anna,2022-08-03 05:35+02:00,2022-08-03 05:35+02:00,2022-08-03 03:35Z,2022-08-03 03:35Z,1,"[Basic, Live]",D-ABYO,3C4B2F,Boeing 747,Lufthansa,NaN
90,LH 591,Expected,IsOperator,False,HKJK,NBO,Nairobi,2022-08-03 05:40+02:00,2022-08-03 05:40+02:00,2022-08-03 03:40Z,2022-08-03 03:40Z,1,"[Basic, Live]",D-AIKL,3C656C,Airbus A330,Lufthansa,NaN
16,LH 1327,Expected,IsOperator,False,DTTA,TUN,Tunis,2022-08-03 05:40+02:00,2022-08-03 05:40+02:00,2022-08-03 03:40Z,2022-08-03 03:40Z,1,"[Basic, Live]",D-AIDH,3C6488,Airbus A321,Lufthansa,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,LH 859,Expected,IsOperator,False,ENGM,OSL,Oslo,2022-08-03 12:00+02:00,2022-08-03 12:00+02:00,2022-08-03 10:00Z,2022-08-03 10:00Z,1,"[Basic, Live]",D-AINY,3C65D9,Airbus A320 NEO,Lufthansa,NaN
74,LH 103,Expected,IsOperator,False,EDDM,MUC,Munich,2022-08-03 12:00+02:00,2022-08-03 12:00+02:00,2022-08-03 10:00Z,2022-08-03 10:00Z,1,"[Basic, Live]",D-AIRW,3C6657,Airbus A321,Lufthansa,NaN
162,LH 231,Expected,IsOperator,False,LIRF,FCO,Rome,2022-08-03 12:00+02:00,2022-08-03 12:00+02:00,2022-08-03 10:00Z,2022-08-03 10:00Z,1,"[Basic, Live]",D-AIZZ,3C675A,Airbus A320 (Sharklets),Lufthansa,NaN
72,LH 393,Expected,IsOperator,False,ELLX,LUX,Luxembourg,2022-08-03 12:00+02:00,2022-08-03 12:00+02:00,2022-08-03 10:00Z,2022-08-03 10:00Z,1,"[Basic, Live]",D-ACNE,3C4DC5,Canadair CRJ 900NG,Lufthansa,NaN
